In [1]:
!pip install -q diffusers transformers accelerate torchvision safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.7 MB/s eta 0:00:00


In [2]:
import torch
from diffusers import DDPMPipeline
from torchvision.utils import save_image
import os
from tqdm import tqdm
from accelerate import Accelerator
import gc

2025-06-04 13:25:25.408443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749043525.595675      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749043525.651990      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 設定資料夾
model_dir = "/kaggle/input/ddpm-pttbeauty-64-v11-linear-80/ddpm-PTTbeauty-64-v11-linear-80/best_model"  # 模型儲存資料夾
output_dir = "/kaggle/working/generated_images_v11-linear-80-300steps"
os.makedirs(output_dir, exist_ok=True)

In [4]:
# 設定參數
num_images = 10000
batch_size = 32
image_size = 64
num_inference_steps = 300

In [5]:
# 初始化 accelerator
accelerator = Accelerator()
device = accelerator.device
print(f"Using device: {device}")
print(f"Number of GPUs available: {torch.cuda.device_count()}")

Using device: cuda
Number of GPUs available: 1


In [6]:
# 初始化 Accelerate
accelerator = Accelerator()
device = accelerator.device

In [7]:
# 載入 pipeline
pipeline = DDPMPipeline.from_pretrained(model_dir, torch_dtype=torch.float16)
pipeline.unet = accelerator.prepare(pipeline.unet)  # 多 GPU 支援

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

The config attributes {'mid_block_type': 'UNetMidBlock2D'} were passed to UNet2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [ ]:
# 計算批次數與索引初始化
n_batches = (num_images + batch_size - 1) // batch_size
global_index = 0

# 開始生成
with accelerator.autocast():
    for i in tqdm(range(n_batches), desc="Generating images"):

        print(f"開始進行第 {i} 個 Epoch")
        
        # 生成圖片
        # images = pipeline(num_inference_steps=num_inference_steps, batch_size=batch_size).images
        images = pipeline(
            num_inference_steps=num_inference_steps,
            batch_size=batch_size,
        ).images
        
        # 儲存圖片
        for j, img in enumerate(images):
            index = global_index + j
            if index >= num_images:
                break
            img.save(os.path.join(output_dir, f"{index:05d}.png"))

        global_index += len(images)

        print(f"已經進行完第 {i} 個 Epoch")

        # 手動清除記憶體
        del images  # 釋放圖片 tensor
        torch.cuda.empty_cache()  # 清除 GPU cache
        gc.collect()  # Python 層垃圾回收

In [9]:
%cd /kaggle/working/generated_images_v11-linear-80-300steps
!zip -rq ../submission_v11-linear-80-300steps.zip .
%cd ..

/kaggle/working/generated_images_v11-linear-80-300steps
/kaggle/working


In [10]:
!python /kaggle/input/fid-test/FID_eval_2.py --img_dir /kaggle/working/generated_images_v11-linear-80-300steps

[INFO] Using GPU: cuda:0
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|█████████████████████████████████████████| 104M/104M [00:00<00:00, 216MB/s]
Extracting features: 100%|████████████████████| 313/313 [00:47<00:00,  6.63it/s]
[RESULT] FID score = 35.9607
